# Coursera Capstone Project - The Battle of the Neighborhoods (Weeks 1 & 2)


# WEEK 1

## Recommending location for opening a gym in the city of Toronto

## Table of contents
* [Aim: Business Problem](#introduction)
* [Data Required](#data)
* [Theory](#methodology)
* [Analysis Procedure](#analysis)
* [Observations and Discussion](#results)
* [Inference](#conclusion)

## Aim: Business Problem <a name="introduction"></a>

_Business problem_  
    We will try to find the best location for opening a gym within the city of Toronto. We will look for places where there is not already a gym. Further, we will prefer locations where people may like to use the gym such as the vicinity of parks where people may come to jog or exercise anyways.  


_Stakeholders (target audience)_  
    This problem is targeted to a franchisee of LA fitness. This person plans to get a franchise of LA fitness to operate the gym. The criteria is to open the gym where there will be minimal competition in the city of Toronto.

## Data Required <a name="data"></a>

We will consider the following factors to base our decision:
* number of existing gyms in the neighborhood
* distance of neighborhood from any parks

Following data sources will be needed to extract/generate the required information:

- location of parks will be got from location data using <b>Foursquare API</b>
- The existence of gyms in neighborhoods will be determined using <b>Foursquare API</b>
- The postal code for neighborhoods will be taken from wikipedia

#### Set up environment

In [21]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Get Data on Canada and filter it down to Toronto city for analysis

In [23]:
with open("canadapostal.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

mytable = soup.table.find_all('td')

pcode = []
borough = []
neigh = []

threeslist = np.arange(0, len(mytable), 3)

for idx in threeslist:
    pcode.append(mytable[idx].text)
    borough.append(mytable[idx + 1].text)
    neigh.append(mytable[idx + 2].text.replace('\n',''))

df_pos = pd.DataFrame(np.array([pcode, borough, neigh])).T

df_pos.columns = ['PostalCode','Borough','Neighborhood']


df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo = df_geo.rename(columns={'Postal Code':'PostalCode'})

df = pd.merge(df_pos, df_geo)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


## Theory <a name="methodology"></a>

We will first start with locating areas in Toronto that have low density of gyms around the center of the city of Toronto.
Next, we will analyze the density of availability of gyms in the different regions of Toronto. 

Then finally, we will pick the best areas within the group of locations chosen. We have to make sure there are not many gyms close by. We will use k-means clustering of those gym locations to get the whereabouts of the final location of choice.


In [24]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index()
toronto_data.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


## Analysis Procedure <a name="analysis"></a>

First, let us look at the Toronto area and get an idea of the number of gyms and their density


### Work with only boroughs that contain the word Toronto 


In [25]:
CLIENT_ID = 'AYOHOCJQIWJOMX4GR0OQ03IROAIVJNIWTSVB3DO2PIOJXPFH' # your Foursquare ID
CLIENT_SECRET = 'K1IPJCQEGWKFZWAHYLJT0QMVV4UGJAYMEYYE3UUBMNXGKIAL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AYOHOCJQIWJOMX4GR0OQ03IROAIVJNIWTSVB3DO2PIOJXPFH
CLIENT_SECRET:K1IPJCQEGWKFZWAHYLJT0QMVV4UGJAYMEYYE3UUBMNXGKIAL


In [26]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [27]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

print(url) # display URL

print()

https://api.foursquare.com/v2/venues/explore?&client_id=AYOHOCJQIWJOMX4GR0OQ03IROAIVJNIWTSVB3DO2PIOJXPFH&client_secret=K1IPJCQEGWKFZWAHYLJT0QMVV4UGJAYMEYYE3UUBMNXGKIAL&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100



In [32]:
results = requests.get(url).json()


In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


### Explore neighborhoods for possibly existence of a Gym

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.01,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,0.010000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.0,0.020000,0.010000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
#         columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
          columns.append('MostCommonVenue{}'.format(ind+1))
    except:
#         columns.append('{}th Most Common Venue'.format(ind+1))
        columns.append('MostCommonVenue{}'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,MostCommonVenue1,MostCommonVenue2,MostCommonVenue3,MostCommonVenue4,MostCommonVenue5,MostCommonVenue6,MostCommonVenue7,MostCommonVenue8,MostCommonVenue9,MostCommonVenue10
0,Adelaide,Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,American Restaurant,Cosmetics Shop,Gym,Hotel,Bakery
1,Bathurst Quay,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry
2,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Cheese Shop,Café,Farmers Market,Steakhouse,Bakery,Seafood Restaurant,Beer Bar
3,Brockton,Coffee Shop,Breakfast Spot,Café,Bakery,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Yoga Studio
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Recording Studio,Smoke Shop,Skate Park,Brewery,Burrito Place,Butcher,Restaurant,Park,Auto Workshop


### Search for competitor gyms

In [73]:
# neighborhoods_venues_sorted[neighborhoods_venues_sorted['MostCommonVenue1'].str.contains('Gym')]
# neighborhoods_venues_sorted[neighborhoods_venues_sorted['MostCommonVenue1'].str.contains('Yoga')]
# neighborhoods_venues_sorted[neighborhoods_venues_sorted['MostCommonVenue2'].str.contains('Gym')]
neighborhoods_venues_sorted[neighborhoods_venues_sorted['MostCommonVenue2'].str.contains('Yoga')]

,Cluster Labels,Neighborhood,MostCommonVenue1,MostCommonVenue2,MostCommonVenue3,MostCommonVenue4,MostCommonVenue5,MostCommonVenue6,MostCommonVenue7,MostCommonVenue8,MostCommonVenue9,MostCommonVenue10
39,0,North Toronto West,Coffee Shop,Yoga Studio,Bagel Shop,Gym / Fitness Center,Clothing Store,Chinese Restaurant,Rental Car Location,Dessert Shop,Diner,Salon / Barbershop


### The  competitions for our gymnasium from the list created is from Neigborhood 4, 13, 39 . We will hence skip those neigborhoods


#### We will now cluster the neighborhoods for like characteristics

In [70]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,MostCommonVenue1,MostCommonVenue2,MostCommonVenue3,MostCommonVenue4,MostCommonVenue5,MostCommonVenue6,MostCommonVenue7,MostCommonVenue8,MostCommonVenue9,MostCommonVenue10
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Café,Gym / Fitness Center
1,3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Café,Gym / Fitness Center
2,13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Fast Food Restaurant,Diner,Italian Restaurant,Japanese Restaurant
3,14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Fast Food Restaurant,Diner,Italian Restaurant,Japanese Restaurant
4,27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Gastropub,Park,Pizza Place,Breakfast Spot


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

In [75]:
c1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
c1.head(3)


,PostalCode,Longitude,Cluster Labels,MostCommonVenue1,MostCommonVenue2,MostCommonVenue3,MostCommonVenue4,MostCommonVenue5,MostCommonVenue6,MostCommonVenue7,MostCommonVenue8,MostCommonVenue9,MostCommonVenue10
0,M5A,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Café,Gym / Fitness Center
1,M5A,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Café,Gym / Fitness Center
2,M5B,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Fast Food Restaurant,Diner,Italian Restaurant,Japanese Restaurant


In [76]:
c2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
c2.head(3)

,PostalCode,Longitude,Cluster Labels,MostCommonVenue1,MostCommonVenue2,MostCommonVenue3,MostCommonVenue4,MostCommonVenue5,MostCommonVenue6,MostCommonVenue7,MostCommonVenue8,MostCommonVenue9,MostCommonVenue10
59,M5V,-79.39442,2,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry
60,M5V,-79.39442,2,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry
61,M5V,-79.39442,2,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry


In [77]:
c3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
c3.head(3)

,PostalCode,Longitude,Cluster Labels,MostCommonVenue1,MostCommonVenue2,MostCommonVenue3,MostCommonVenue4,MostCommonVenue5,MostCommonVenue6,MostCommonVenue7,MostCommonVenue8,MostCommonVenue9,MostCommonVenue10
49,M4T,-79.38316,3,Playground,Tennis Court,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
50,M4T,-79.38316,3,Playground,Tennis Court,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


## Observations and Discussion <a name="results"></a>

Based on our analysis, we were able to find good locations where the density of availability of gyms is quite low. We evaluated the assumption that parks and such areas conducive to health related activities are better suited for opening a new gym in the vicinity.

We then laid a high level grid searching to identify our starting point of analysis. We then narrowed it down to a smaller radius of search where any incident of a gym within a 500 m radius was eliminated.

Using k-means clustering, we created zones where the opening of a gym was best suited. Once done, we ended up with zones that had the best potential for a gym that was in the vicinity to a health center. 

This recommendation should be discussed with the stakeholder to ensure it conforms to their standards. The multiple recommendation of locations should help with providing enough choices for the stakehold to make the final decision.


## Inference <a name="conclusion"></a>

The aim here was to find out the best locations for opening a new gym in the Toronto city area. The caveats used were to look for places where there would be minimal competition and also to identify locations that are health friendly, such as parks. We located a collection of locations based on the requirements stated above. These locations were then clustered to create areas of best potential. These were then used for recommendation to the stakeholders.

The reports created will be presented to the stakeholders for final selection from the recommendations provided out of our analysis. For future analysis, factors beyond health conditions and gym density may be considered.
